# Linear properties of split-and-lookup S-boxes

## Linear approximations

In [ ]:
# Get the approximations of small S-boxes

def gen_lis(L, a, b, vals):
    """
    Generate a table:
    * keys are the values achieved by the map a*L(x)-b*x where x in vals
    * values are lists of x's achieving this value
    """
    lis = {}
    
    for x in vals:
        y = a*L[x] - b*x
        if y in lis.keys():
            lis[y] += 1
        else:
            lis[y] = 1
    return lis

def get_count(c, lis):
    """
    Get the number of x such that a*L(x)-b*x = c.
    """
    if c in lis.keys():
        return lis[c]
    else:
        return 0


In [ ]:
# Generate the carry matrices

def const_to_mat(lis, a, b, c, s):
    """
    Compute the matrix whose coefficients are
    * m_ij = #{x | a*L(x)-b*x = c+i*s - j}
    """
    low = -(max(0, -a) + max(0,  b) + 1)
    up  =   max(0,  a) + max(0, -b)
    
    mat = matrix(ZZ, [[get_count(c-i+j*s, lis) for i in range(low, up+1)]
                      for j in range(low, up+1)])
    
    return mat


def list_to_mat(lis, a, b, c_list, s_list):
    """
    Compute the same matrix, but for a list of constants using matrix product.
    """
    n = len(c_list)
    
    mat = const_to_mat(lis, a, b, c_list[0], s_list[0])
    for i in range(1, n):
        mat = mat * const_to_mat(lis, a, b, c_list[i], s_list[i])
    return mat

In [ ]:
# Transform a field element into a list

def int_to_list(c, s_list):
    """
    Transform a Fp field element into a list of integers
    Decomposition in base s.
    """
    c_list = []
    n = len(s_list)
    
    for i in range(n):
        c_list.append(c % s_list[n-i-1])
        c //= s_list[n-i-1]
    c_list.reverse()
    return c_list

In [ ]:
# Compute the number of approximations.

def mat(L, a, b, c, s_list, v_list):
    """
    Generate the carry matrix of S.
    """
    n = len(s_list)
    
    c_list = int_to_list(c, s_list)
    lis_list = []
    lis_list_f = []
    lis_list_v = []
    
    for i in range(n):
        lis_list.append(gen_lis(L, a, b, range(s_list[i])))
        lis_list_f.append(gen_lis(L, a, b, range(v_list[i])))
        lis_list_v.append(gen_lis(L, a, b, [v_list[i]]))
    
    mat_list_cum = [const_to_mat(lis_list[n-1], a, b, c_list[n-1], s_list[n-1])]
    mat_list_f = []
    mat_list_v_cum = [const_to_mat(lis_list_v[0], a, b, c_list[0], s_list[0])]
    
    for i in range(n):
        mat_list_f.append(const_to_mat(lis_list_f[i], a, b, c_list[i], s_list[i]))
    
    for i in range(1,n):
        mat_list_cum.append(const_to_mat(lis_list[n-i-1], a, b, c_list[n-i-1], s_list[n-i-1])*mat_list_cum[-1])
        mat_list_v_cum.append(mat_list_v_cum[-1]*const_to_mat(lis_list_v[i], a, b, c_list[i], s_list[i]))
    
    mat_list_cum.reverse()
    
    mat = mat_list_f[0]*mat_list_cum[1]
    
    for i in range(n-2):
        mat += mat_list_v_cum[i]*mat_list_f[i+1]*mat_list_cum[i+2]
        
    mat += mat_list_v_cum[n-2]*mat_list_f[n-1]
    mat += mat_list_v_cum[n-1]
    
    return mat

def approximations(L, a, b, c, s_list, p):
    """
    Compute the number of approximations of the form:
    S(x) = b/a *x + c
    """
    v_list = int_to_list(p-1, s_list)
    
    s_prod = 1
    
    for si in s_list:
        s_prod *= si
    
    low = -(max(0, -a) + max(0,  b) + 1)
    up  =   max(0,  a) + max(0, -b)
    
    score = 0
    
    for k in range(low, up+1):
        c_prime  = (c + k*p)
        k_prime  = c_prime // s_prod
        c_prime %= s_prod
        
        mat_k = mat(L, a, b, c_prime, s_list, v_list)
        
        score += mat_k[k_prime -low, -low]
        
    return score

In [ ]:
# Case where p is the Mersenne prime with two different kinds of S-boxes

def mat_mersenne(L256, L128, a, b, c):
    """
    Generate the carry matrix of S in the case where p is the Mersenne prime.
    """
    lis256 = gen_lis(L256, a, b, range(256))
    lis255 = gen_lis(L256, a, b, [255])
    lis128 = gen_lis(L128, a, b, range(128))
    lis127 = gen_lis(L128, a, b, [127])
    
    s_list = [128, 256, 256, 256]
    
    c_list = int_to_list(c, s_list)
    mat_full = const_to_mat(lis128, a, b, c_list[0], s_list[0])*list_to_mat(lis256, a, b, c_list[1:], s_list[1:])
    mat_ffff = const_to_mat(lis127, a, b, c_list[0], s_list[0])*list_to_mat(lis255, a, b, c_list[1:], s_list[1:])
    
    mat = mat_full - mat_ffff
    return mat
   
def approximations_mersenne(L256, L128, a, b, c):
    """
    Compute the number of approximations in the case where p is the Mersenne prime.
    """
    mat = mat_mersenne(L256, L128, a, b, c)
    return mat.trace()

# Linear correlation

In [ ]:
def summation(L, a, b, s_cur, vals, p):
    """
    Sum over a single small S-box.
    """
    res = 0
    
    zeta_p = CC.zeta(p)
    
    for x in vals:
        res += zeta_p**((a*L[x]-b*x)*s_cur % p)
    return res


def walsh_coeff(L, a, b, s_list, p):
    """
    Compute a coefficient of the Walsh transform of S.
    """
    v_list = int_to_list(p-1, s_list)
    n = len(s_list)
    
    sum_list = []
    sum_list_f = []
    sum_list_v = []
    
    s_cur = 1
    
    for i in range(n):
        s_cur  *= s_list[i]
        
    for i in range(n):
        s_cur //= s_list[i]
        sum_list.append(summation(L, a, b, s_cur, range(s_list[i]), p))
        sum_list_f.append(summation(L, a, b, s_cur, range(v_list[i]), p))
        sum_list_v.append(summation(L, a, b, s_cur, [v_list[i]], p))
    
    sum_list_cum = [sum_list[n-1]]
    sum_list_v_cum = [sum_list_v[0]]
    
    for i in range(1,n):
        sum_list_cum.append(sum_list[n-i-1]*sum_list_cum[-1])
        sum_list_v_cum.append(sum_list_v_cum[-1]*sum_list_v[i])
    
    sum_list_cum.reverse()
    
    res = sum_list_f[0]*sum_list_cum[1]
    
    for i in range(n-2):
        res += sum_list_v_cum[i]*sum_list_f[i+1]*sum_list_cum[i+2]
        
    res += sum_list_v_cum[n-2]*sum_list_f[n-1]
    res += sum_list_v_cum[n-1]
    
    return res

In [ ]:
def walsh_coeff_mers(L256, L128, a, b):
    """
    Compute a coefficient of the Walsh transform of S
    in the case p = 2^31 - 1.
    """
    sum_list = 1
    p_mers = 2**31-1
    
    for i in range(3):
        sum_list *= summation(L256, a, b, 2**(8*i), range(256), p_mers)
    sum_list *= summation(L128, a, b, 2**24, range(128), p_mers)
    
    # remove the value for x = 2^31-1 (which is always 1).
    sum_list -= 1
    return sum_list

# Some examples of split-and-lookups

In [ ]:
s_goldi = [2**8 for i in range(8)]
p_goldi = 2**64 -2**32 +1

p_mers = 2**31 -1

### AES S-box (Rescaled)

In [ ]:
from sage.crypto.sboxes import sboxes

F28.<x> = GF(2)[x].quotient(x**8 + x**4 + x**3 + x + 1)

def int_to_F28(xi):
    res = F28(0)
    
    for i in range(8):
        res += (xi % 2) *x**i
        xi//= 2
    return res

def F28_to_int(xi):
    res = 0
    
    for i in range(len(xi.list())):
        res += int(xi.list()[i])*2**i
    return res


In [ ]:
# Get AES S-box
L_aes = sboxes["AES"]

# Add fixed points 0x00 and 0xff
L_aes = [int_to_F28(y ^^ 0x63) for y in L_aes]
L_aes = [y*int_to_F28(0xff)/int_to_F28(0x75) for y in L_aes]
L_aes = [F28_to_int(y) for y in L_aes]

print("AES S-box:")
print(L_aes)


### iScream S-box (Rescaled)

In [ ]:
# Get iScream S-box
L_iscream = sboxes["iScream"]

# Add fixed point 0xff
L_iscream = [y ^^ (y << 1) & 0xff for y in L_iscream]
L_iscream = [y ^^ (y << 4) & 0xff for y in L_iscream]

print("iScream S-box:")
print(L_iscream)

### Tip5 S-box

In [ ]:
Fr = GF(257)

In [ ]:
L_tip5 = [(Fr(x)+1)**3-1 for x in range(256)]
L_tip5 = [int(y) for y in L_tip5]

print("Tip5 S-box:")
print(L_tip5)

### Monolith S-box

In [ ]:
def shift(x, sh, n):
    return ((x  << sh) | (x >> (n-sh))) & (2**n-1)

def not_(x, n):
    return x ^^ (2**n -1)

def chi_daemen_shift(x):
    return shift(x ^^ (shift(not_(x, 8), 1, 8) & shift(x, 2, 8) & shift(x, 3, 8)), 1, 8)

def chi_daemen_shift_7(x):
    return shift(x ^^ (shift(not_(x, 7), 1, 7) & shift(x, 2, 7)), 1, 7)

In [ ]:
L_monolith   = [chi_daemen_shift(x) for x in range(256)]
L_monolith_7 = [chi_daemen_shift_7(x) for x in range(128)]

print("Monolith S-box:")
print(L_monolith)

print("Monolith S-box (7 bits):")
print(L_monolith_7)

### Reinforced concrete S-boxes

In [ ]:
p_BLS381 = 0x73eda753299d7d483339d80809a1d80553bda402fffe5bfeffffffff00000001

p_prime_BLS381 = 659

s_BLS381 = [679, 703, 688, 691, 702, 703, 697, 
         698, 695, 701, 701, 701, 699, 694, 
         701, 694, 700, 688, 700, 693, 691, 
         695, 679, 668, 694, 696, 693]


In [ ]:
L_BLS381 = [
        171, 178, 483, 527, 653, 408, 197, 599, 300, 607, 403, 511, 579, 520, 591, 412, 261, 559,
        551, 154, 180, 138, 596, 150, 276, 271, 48, 168, 362, 637, 467, 164, 536, 554, 287, 530,
        431, 92, 654, 518, 323, 572, 624, 4, 258, 439, 430, 495, 534, 222, 545, 31, 44, 18, 80, 55,
        399, 328, 505, 313, 441, 586, 501, 598, 566, 568, 77, 496, 106, 563, 537, 78, 50, 450, 445,
        166, 237, 617, 185, 404, 621, 578, 133, 517, 646, 98, 86, 492, 267, 193, 33, 476, 207, 17,
        487, 643, 52, 384, 74, 148, 121, 657, 633, 528, 269, 611, 567, 601, 391, 231, 226, 658,
        331, 191, 354, 23, 474, 277, 390, 341, 279, 442, 422, 638, 15, 196, 329, 377, 36, 433, 398,
        72, 256, 352, 253, 550, 635, 142, 343, 176, 500, 588, 413, 569, 266, 42, 283, 535, 410,
        538, 647, 85, 27, 423, 558, 61, 356, 348, 43, 19, 625, 291, 238, 274, 432, 448, 100, 642,
        260, 587, 622, 608, 366, 420, 477, 316, 605, 254, 130, 407, 471, 174, 631, 34, 652, 628,
        175, 134, 122, 192, 531, 217, 32, 257, 145, 307, 262, 83, 509, 440, 600, 589, 359, 522,
        268, 143, 498, 512, 333, 651, 151, 183, 126, 351, 39, 246, 242, 630, 543, 574, 610, 655,
        25, 494, 456, 612, 123, 315, 340, 296, 580, 503, 281, 428, 62, 10, 76, 203, 288, 91, 426,
        128, 629, 29, 218, 292, 447, 161, 117, 388, 540, 364, 245, 541, 224, 502, 370, 229, 90,
        466, 636, 208, 51, 562, 259, 344, 334, 111, 235, 488, 632, 577, 54, 386, 75, 181, 463, 421,
        24, 96, 406, 156, 158, 265, 5, 310, 37, 124, 88, 155, 480, 593, 202, 451, 1, 497, 645, 457,
        187, 56, 206, 179, 640, 249, 99, 240, 460, 490, 163, 369, 293, 186, 553, 46, 449, 41, 219,
        308, 7, 234, 336, 373, 372, 347, 215, 481, 542, 146, 357, 656, 136, 330, 595, 516, 592,
        273, 365, 8, 47, 641, 81, 484, 573, 614, 437, 533, 0, 282, 184, 400, 49, 114, 374, 280,
        499, 418, 139, 382, 613, 233, 345, 393, 575, 508, 299, 101, 582, 360, 285, 2, 376, 548,
        189, 648, 214, 618, 385, 371, 425, 552, 204, 286, 443, 210, 294, 211, 241, 461, 275, 165,
        350, 59, 583, 159, 434, 252, 71, 436, 529, 236, 475, 339, 367, 147, 170, 110, 22, 298, 506,
        172, 247, 513, 73, 230, 314, 239, 157, 116, 65, 11, 570, 40, 620, 205, 251, 594, 468, 69,
        489, 109, 452, 465, 312, 383, 129, 379, 335, 353, 602, 546, 243, 57, 473, 486, 320, 162,
        526, 115, 26, 560, 107, 458, 519, 169, 97, 358, 504, 414, 13, 459, 132, 167, 402, 14, 491,
        571, 105, 112, 363, 581, 194, 84, 349, 201, 462, 289, 53, 603, 209, 396, 303, 317, 102, 82,
        131, 639, 3, 435, 378, 415, 539, 223, 30, 510, 199, 479, 397, 45, 248, 561, 67, 213, 438,
        20, 405, 557, 120, 89, 584, 555, 264, 419, 525, 429, 392, 311, 68, 446, 270, 585, 113, 627,
        472, 38, 375, 327, 127, 417, 547, 12, 108, 368, 95, 250, 322, 198, 380, 149, 104, 87, 332,
        135, 28, 318, 482, 221, 188, 58, 544, 521, 93, 324, 64, 272, 297, 644, 453, 225, 606, 295,
        216, 152, 411, 361, 444, 469, 427, 507, 395, 609, 153, 381, 464, 424, 94, 9, 564, 321, 615,
        21, 227, 137, 70, 326, 549, 556, 565, 416, 470, 255, 60, 604, 590, 305, 35, 278, 6, 125,
        387, 220, 597, 63, 454, 401, 119, 302, 309, 342, 16, 619, 493, 290, 616, 173, 304, 195,
        524, 263, 212, 649, 626, 409, 338, 306, 389, 79, 160, 66, 177, 232, 478, 514, 650, 455,
        103, 144, 355, 182, 346, 284, 200, 634, 244, 140, 337, 325, 319, 532, 394, 118, 485, 301,
        623, 190, 523, 515, 576, 141, 228
    ] + list(range(p_prime_BLS381, max(s_BLS381)))


In [ ]:
p_BN254 = 0x30644e72e131a029b85045b68181585d2833e84879b9709143e1f593f0000001

p_prime_BN254 = 641

s_BN254 = [673, 678, 667, 683, 680, 655, 683, 683, 681,
          683, 675, 668, 675, 677, 680, 681, 669, 683, 
           681, 677, 668, 654, 663, 666, 656, 658, 651]

In [ ]:
L_BN254 = [
        377, 222, 243, 537, 518, 373, 152, 435, 526, 352, 2, 410, 513, 545, 567, 354, 405, 80, 233,
        261, 49, 240, 568, 74, 131, 349, 146, 278, 330, 372, 43, 432, 247, 583, 105, 203, 637, 307,
        29, 597, 633, 198, 519, 95, 148, 62, 68, 312, 616, 357, 234, 433, 154, 90, 163, 249, 101,
        573, 447, 587, 494, 103, 608, 394, 409, 73, 317, 305, 346, 562, 262, 313, 303, 550, 64,
        102, 259, 400, 495, 572, 238, 40, 612, 236, 586, 15, 361, 386, 138, 136, 107, 33, 190, 423,
        176, 161, 460, 35, 202, 589, 32, 160, 444, 517, 490, 515, 144, 195, 269, 332, 25, 308, 192,
        276, 623, 180, 626, 217, 329, 66, 392, 431, 12, 478, 67, 232, 258, 355, 94, 191, 632, 181,
        298, 1, 301, 79, 618, 523, 627, 484, 306, 610, 635, 619, 544, 420, 408, 158, 328, 61, 406,
        299, 442, 178, 625, 621, 497, 465, 574, 143, 54, 57, 89, 322, 135, 96, 605, 599, 473, 97,
        85, 133, 200, 93, 291, 525, 529, 206, 614, 319, 196, 482, 17, 168, 70, 104, 441, 159, 364,
        603, 78, 150, 230, 116, 31, 630, 132, 69, 499, 532, 218, 492, 112, 505, 437, 333, 457, 456,
        439, 639, 398, 16, 436, 264, 450, 211, 241, 524, 294, 235, 126, 165, 527, 452, 212, 157,
        272, 208, 469, 611, 338, 83, 326, 151, 139, 607, 285, 585, 58, 14, 193, 71, 440, 511, 542,
        390, 470, 155, 413, 606, 142, 367, 371, 174, 5, 60, 289, 297, 336, 370, 76, 209, 622, 453,
        257, 555, 44, 430, 345, 335, 548, 459, 47, 426, 591, 559, 417, 284, 552, 137, 277, 281,
        463, 631, 350, 265, 323, 108, 290, 169, 634, 609, 414, 130, 6, 166, 316, 207, 592, 280,
        391, 274, 20, 300, 593, 549, 3, 602, 418, 472, 419, 296, 41, 46, 615, 638, 388, 553, 282,
        356, 327, 462, 115, 325, 121, 399, 273, 334, 383, 488, 292, 55, 628, 9, 19, 601, 496, 228,
        201, 576, 374, 558, 153, 162, 341, 353, 84, 220, 461, 221, 547, 344, 507, 577, 140, 485,
        471, 11, 175, 13, 53, 543, 270, 120, 30, 584, 384, 368, 397, 239, 4, 483, 620, 189, 522,
        540, 510, 149, 245, 533, 283, 256, 369, 302, 571, 128, 253, 448, 446, 183, 99, 438, 468,
        42, 594, 487, 403, 23, 172, 340, 106, 481, 251, 363, 295, 489, 474, 337, 87, 86, 246, 215,
        376, 315, 415, 117, 286, 600, 56, 145, 91, 358, 429, 411, 516, 310, 213, 598, 10, 395, 111,
        506, 237, 170, 512, 82, 147, 579, 402, 501, 343, 38, 434, 214, 314, 360, 77, 565, 320, 385,
        404, 199, 331, 351, 466, 596, 365, 231, 477, 604, 254, 268, 539, 424, 167, 378, 491, 535,
        141, 267, 177, 27, 546, 219, 556, 216, 451, 387, 28, 50, 569, 255, 288, 156, 449, 379, 508,
        528, 531, 624, 581, 554, 59, 171, 252, 0, 595, 185, 51, 520, 575, 475, 113, 187, 194, 428,
        500, 617, 188, 321, 179, 263, 110, 467, 18, 401, 22, 164, 342, 21, 382, 381, 127, 52, 570,
        45, 445, 36, 534, 339, 98, 293, 244, 266, 629, 229, 122, 123, 48, 88, 225, 173, 100, 114,
        536, 636, 205, 34, 425, 502, 514, 304, 613, 530, 118, 75, 561, 582, 81, 480, 92, 498, 464,
        224, 479, 563, 223, 640, 521, 427, 503, 250, 375, 186, 72, 242, 125, 380, 271, 204, 407,
        366, 197, 119, 7, 493, 26, 109, 65, 359, 396, 311, 309, 458, 134, 393, 557, 476, 324, 421,
        275, 37, 39, 580, 184, 560, 8, 455, 509, 422, 24, 287, 590, 182, 416, 318, 260, 578, 454,
        389, 129, 566, 63, 486, 541, 362, 210, 551, 348, 279, 538, 347, 504, 124, 564, 443, 412,
        226, 227, 248, 588
    ] + list(range(p_prime_BN254, max(s_BN254)))


In [ ]:
p_ST = 0x3fa000000000000000000000000000000000000000000000000000000000001

p_prime_ST = 1013

s_ST = [1024 for i in range(24)]+[1023]


In [ ]:
L_ST = [
        849, 68, 27, 909, 988, 687, 828, 507, 847, 380, 656, 379, 340, 296, 974, 3, 338, 355, 263,
        968, 754, 119, 442, 231, 629, 634, 938, 484, 73, 954, 704, 20, 1006, 447, 977, 591, 528,
        593, 103, 69, 236, 45, 843, 461, 762, 158, 908, 661, 751, 874, 545, 96, 35, 802, 738, 495,
        597, 560, 956, 518, 262, 991, 54, 156, 821, 646, 620, 581, 454, 470, 753, 617, 550, 91,
        647, 481, 475, 992, 287, 141, 523, 7, 233, 51, 89, 614, 336, 126, 857, 882, 194, 806, 55,
        793, 443, 584, 213, 967, 110, 673, 645, 979, 446, 116, 621, 795, 760, 1, 473, 543, 185,
        1008, 399, 105, 344, 205, 914, 830, 851, 927, 393, 290, 716, 17, 906, 170, 918, 895, 638,
        26, 327, 409, 161, 371, 559, 363, 513, 67, 61, 121, 549, 886, 62, 822, 925, 747, 357, 618,
        201, 624, 464, 665, 892, 317, 302, 943, 235, 64, 642, 416, 104, 799, 521, 839, 875, 220,
        623, 921, 361, 522, 234, 625, 562, 128, 800, 117, 275, 81, 90, 313, 834, 176, 554, 82, 168,
        928, 504, 637, 764, 721, 532, 193, 100, 911, 434, 890, 28, 160, 565, 541, 397, 901, 996,
        23, 922, 146, 301, 844, 303, 697, 107, 136, 768, 869, 494, 347, 428, 798, 949, 957, 735,
        929, 0, 294, 619, 677, 299, 548, 8, 43, 284, 202, 232, 260, 109, 745, 982, 976, 78, 695,
        845, 790, 826, 375, 755, 524, 823, 450, 512, 1000, 540, 948, 856, 568, 269, 712, 771, 873,
        816, 999, 195, 811, 199, 708, 348, 539, 765, 133, 774, 500, 49, 492, 24, 162, 891, 211,
        258, 582, 729, 346, 18, 788, 360, 217, 820, 448, 249, 1010, 405, 316, 430, 228, 410, 803,
        692, 4, 852, 224, 777, 752, 22, 950, 455, 883, 97, 557, 488, 221, 585, 124, 879, 342, 458,
        981, 670, 827, 387, 219, 120, 858, 930, 414, 932, 411, 207, 558, 123, 696, 47, 369, 920,
        813, 351, 503, 809, 343, 268, 664, 505, 118, 70, 970, 30, 324, 325, 863, 570, 987, 789, 76,
        936, 903, 190, 218, 401, 706, 2, 276, 514, 632, 247, 705, 805, 586, 794, 993, 32, 34, 15,
        502, 84, 672, 214, 733, 984, 417, 724, 72, 866, 66, 842, 685, 717, 297, 469, 668, 636, 192,
        12, 145, 1003, 627, 700, 756, 281, 635, 385, 783, 893, 298, 11, 251, 131, 819, 931, 31,
        641, 285, 429, 178, 19, 868, 186, 792, 530, 689, 106, 366, 730, 169, 739, 10, 538, 872,
        796, 786, 39, 833, 273, 563, 271, 200, 453, 283, 825, 462, 1007, 657, 727, 139, 419, 280,
        740, 720, 898, 889, 510, 832, 423, 383, 256, 942, 33, 841, 613, 319, 471, 48, 779, 406,
        198, 564, 924, 465, 770, 650, 535, 413, 330, 590, 933, 1001, 734, 651, 432, 534, 436, 486,
        876, 111, 596, 345, 531, 177, 41, 95, 245, 552, 606, 653, 743, 667, 837, 767, 138, 744,
        203, 659, 307, 648, 723, 726, 569, 997, 980, 44, 154, 227, 797, 499, 881, 153, 609, 382,
        511, 812, 763, 439, 216, 125, 323, 566, 900, 517, 818, 305, 814, 293, 400, 728, 829, 815,
        241, 854, 592, 304, 913, 261, 407, 370, 533, 703, 403, 372, 761, 229, 75, 587, 150, 669,
        575, 252, 985, 463, 164, 438, 542, 181, 516, 526, 288, 306, 680, 595, 248, 556, 425, 959,
        426, 386, 519, 311, 561, 38, 995, 690, 746, 191, 182, 758, 679, 962, 896, 772, 130, 877,
        870, 951, 536, 701, 732, 134, 553, 114, 490, 989, 174, 242, 860, 940, 941, 939, 574, 736,
        601, 420, 389, 71, 888, 183, 359, 986, 332, 681, 451, 567, 412, 267, 188, 675, 58, 576,
        850, 817, 722, 894, 77, 152, 395, 112, 944, 750, 551, 277, 135, 254, 240, 459, 189, 196,
        684, 259, 643, 87, 333, 132, 741, 749, 74, 376, 115, 907, 599, 364, 92, 171, 6, 508, 244,
        600, 79, 952, 824, 209, 477, 631, 958, 710, 501, 80, 694, 660, 1011, 14, 289, 52, 537, 808,
        339, 801, 836, 445, 766, 963, 384, 546, 250, 615, 965, 785, 487, 686, 640, 468, 264, 698,
        525, 910, 945, 489, 926, 418, 579, 961, 731, 328, 835, 611, 318, 13, 616, 255, 855, 865,
        341, 444, 312, 737, 257, 491, 529, 639, 279, 326, 662, 682, 496, 804, 482, 983, 583, 960,
        775, 572, 424, 916, 5, 603, 711, 757, 335, 912, 396, 278, 295, 791, 848, 947, 781, 172,
        972, 282, 50, 699, 971, 719, 184, 633, 449, 966, 25, 147, 381, 368, 197, 274, 742, 655,
        649, 955, 748, 1004, 605, 588, 905, 969, 457, 493, 978, 309, 666, 479, 885, 21, 140, 973,
        917, 644, 923, 466, 871, 678, 476, 769, 626, 472, 573, 934, 437, 300, 878, 374, 98, 688,
        149, 485, 838, 143, 594, 57, 1005, 1012, 919, 179, 456, 377, 773, 349, 85, 880, 718, 243,
        433, 166, 520, 93, 408, 807, 715, 83, 390, 334, 467, 864, 398, 337, 155, 602, 352, 388,
        173, 440, 208, 391, 422, 709, 612, 331, 266, 497, 707, 270, 431, 460, 478, 598, 810, 483,
        392, 498, 350, 129, 365, 713, 953, 246, 610, 904, 787, 663, 435, 29, 226, 127, 902, 452,
        759, 652, 215, 506, 782, 362, 676, 37, 674, 265, 9, 831, 163, 862, 180, 671, 358, 167, 42,
        238, 60, 780, 272, 59, 315, 314, 946, 658, 94, 367, 322, 884, 113, 175, 210, 122, 212, 607,
        555, 310, 994, 329, 778, 204, 63, 714, 1002, 101, 40, 1009, 630, 474, 142, 225, 230, 165,
        292, 404, 702, 846, 693, 421, 157, 102, 108, 137, 515, 86, 222, 580, 402, 577, 36, 353,
        148, 46, 320, 975, 887, 776, 990, 628, 683, 151, 56, 544, 937, 480, 604, 571, 159, 622,
        356, 861, 53, 99, 608, 589, 65, 784, 691, 239, 867, 853, 441, 859, 998, 237, 527, 899, 725,
        394, 373, 16, 223, 253, 354, 509, 378, 578, 187, 291, 308, 415, 964, 427, 915, 547, 144,
        897, 935, 88, 840, 286, 206, 321, 654
    ] + list(range(p_prime_ST, max(s_ST)))


### Reinforced concrete (challenge versions)

In [ ]:
p_easy = 281474976710597

s_easy = [267, 267, 267, 244, 258, 235]

p_prime_easy = 223

L_easy = [int(GF(p_prime_easy)(x)**(p_prime_easy-2)) for x in range(p_prime_easy)] + [x for x in range(p_prime_easy, max(s_easy))]

In [ ]:
p_medium = 72057594037926839

s_medium = [638, 659, 635, 646, 659, 634]

p_prime_medium = 617

L_medium = [int(GF(p_prime_medium)(x)**(p_prime_medium-2)) for x in range(p_prime_medium)] + [x for x in range(p_prime_medium, max(s_medium))]

In [ ]:
p_hard = 18446744073709551557

s_hard = [570, 577, 549, 579, 553, 577, 553]

p_prime_hard = 541

L_hard = [int(GF(p_prime_hard)(x)**(p_prime_hard-2)) for x in range(p_prime_hard)] + [x for x in range(p_prime_hard, max(s_hard))]

## Test linear approximations

In [ ]:
a, b, c = 1, 1, 2025524839466146844

print("Best approximation found for AES: {}*S(x) = {}*x + {}".format(a, b, c))
print("For {} values of x.".format(approximations(L_aes, a, b, c, s_goldi, p_goldi)))

In [ ]:
a, b, c = 1, -1, 1157442769704194062

print("Best approximation found for iScream: {}*S(x) = {}*x + {}".format(a, b, c))
print("For {} values of x.".format(approximations(L_iscream, a, b, c, s_goldi, p_goldi)))

In [ ]:
a, b, c = 1, -1, 606306544323790996

print("Best approximation found for Tip5: {}*S(x) = {}*x + {}".format(a, b, c))
print("For {} values of x.".format(approximations(L_tip5, a, b, c, s_goldi, p_goldi)))

In [ ]:
a, b, c = 1, 2, 0

print("Best approximation found for Monolith-64: {}*S(x) = {}*x + {}".format(a, b, c))
print("For {} values of x.\n".format(approximations(L_monolith, a, b, c, s_goldi, p_goldi)))

a, b, c = 1, 2, 2**24

print("Best approximation found for Monolith-31: {}*S(x) = {}*x + {}".format(a, b, c))
print("For {} values of x.".format(approximations_mersenne(L_monolith, L_monolith_7, a, b, c)))

In [ ]:
# Fixed points of the three instanciations of Reinforced concrete.

print("Fixed points for Reinforced concrete-BLS381:")
print(approximations(L_BLS381, 1, 1, 0, s_BLS381, p_BLS381))

print("\nFixed points for Reinforced concrete-BN254:")
print(approximations(L_BN254, 1, 1, 0, s_BN254, p_BN254))

print("\nFixed points for Reinforced concrete-ST:")
print(approximations(L_ST, 1, 1, 0, s_ST, p_ST))

In [ ]:
# Fixed points of the challenge instanciations of Reinforced concrete.

print("Fixed points for Reinforced concrete-easy:")
print(approximations(L_easy, 1,1,0,s_easy, p_easy))

print("\nFixed points for Reinforced concrete-medium:")
print(approximations(L_medium, 1,1,0,s_medium, p_medium))

print("\nFixed points for Reinforced concrete-hard:")
print(approximations(L_hard, 1,1,0,s_hard, p_hard))

## Test linear correlation

In [ ]:
a, b = 22, 14

print("Best correlation found for Tip5:")
print("{}*S(x) - {}*x : {}".format(a, b, abs(walsh_coeff(L_tip5, a, b, s_goldi, p_goldi))/p_goldi))

In [ ]:
a, b = 1, 2

print("Best correlation found for Monolith-64:")
print("{}*S(x) - {}*x : {}".format(a, b, abs(walsh_coeff(L_monolith, a, b, s_goldi, p_goldi))/p_goldi))

a, b = 128, 256

print("\nBest correlation found for Monolith-31:")
print("{}*S(x) - {}*x : {}".format(a, b, abs(walsh_coeff_mers(L_monolith, L_monolith_7, a, b))/p_mers))

In [ ]:
a, b = 4, 6

print("Best correlation found for Reinforced Concrete-BLS381:")
print("{}*S(x) - {}*x : {}\n".format(a, b, abs(walsh_coeff(L_BLS381, a, b, s_BLS381, p_BLS381))/p_BLS381))

a, b = 5, 7

print("Best correlation found for Reinforced Concrete-BN254:")
print("{}*S(x) - {}*x : {}\n".format(a, b, abs(walsh_coeff(L_BN254, a, b, s_BN254, p_BN254))/p_BN254))

a, b = 4, -1

print("Best correlation found for Reinforced Concrete-ST:")
print("{}*S(x) + {}*x : {}".format(a, -b, abs(walsh_coeff(L_ST, a, b, s_ST, p_ST))/p_ST))

In [ ]:
a, b = 18, 17

print("Best correlation found for Reinforced Concrete-easy:")
print("{}*S(x) - {}*x : {}\n".format(a, b, abs(walsh_coeff(L_easy, a, b, s_easy, p_easy))/p_easy))

a, b = 13, 15

print("Best correlation found for Reinforced Concrete-medium:")
print("{}*S(x) - {}*x : {}\n".format(a, b, abs(walsh_coeff(L_medium, a, b, s_medium, p_medium))/p_medium))

a, b = 11, 14

print("Best correlation found for Reinforced Concrete-hard:")
print("{}*S(x) - {}*x : {}\n".format(a, b, abs(walsh_coeff(L_hard, a, b, s_hard, p_hard))/p_hard))